In [1]:
# Load API_key from .env file
from config_loader import ConfigLoader
config = ConfigLoader()

In [2]:
from typing import Annotated

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages
from langchain_core.messages import AIMessage
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.checkpoint.memory import MemorySaver

def select_model():
    choice_map = {
        "1": ("Claude", ChatAnthropic(model="claude-3-5-sonnet-20240620")),
        "2": ("GPT-3", ChatOpenAI(model_name="gpt-3.5-turbo-1106"))
    }

    while (choice := input("Enter 1 for Claude or 2 for GPT-3: ").strip()) not in choice_map:
        print("Invalid choice. Please enter 1 or 2.")

    model_name, llm_instance = choice_map[choice]
    print(f"You selected {model_name}.")
    return llm_instance

memory = MemorySaver()

class State(TypedDict):
    messages: Annotated[list, add_messages]    

graph_builder = StateGraph(State)

#Add New function; Tools
tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = select_model()
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

#This is new
graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_before=["tools"],
    # interrupt_after=["tools"]
)

Enter 1 for Claude or 2 for GPT-3:  1


You selected Claude.


In [3]:
user_input = "I'm learning LangGraph. Could you do some research on it for me?"
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================

[{'text': "Certainly! I'd be happy to research LangGraph for you. To get the most up-to-date and comprehensive information, I'll use the Tavily search engine to look this up. Let me do that for you now.", 'type': 'text'}, {'id': 'toolu_01EKxe7qtC1ta3XokejDoryQ', 'input': {'query': 'LangGraph framework for building language model applications'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01EKxe7qtC1ta3XokejDoryQ)
 Call ID: toolu_01EKxe7qtC1ta3XokejDoryQ
  Args:
    query: LangGraph framework for building language model applications


In [6]:
#현재 graph의 어느 위치에 있는지 확인
snapshot = graph.get_state(config)
#자이제 다음단계가 'chatbot'임을 알려줌
snapshot.next

('tools',)

In [7]:
existing_message = snapshot.values["messages"][-1]
print(existing_message)
existing_message.tool_calls

content=[{'text': "Certainly! I'd be happy to research LangGraph for you. To get the most up-to-date and comprehensive information, I'll use the Tavily search engine to look this up. Let me do that for you now.", 'type': 'text'}, {'id': 'toolu_01EKxe7qtC1ta3XokejDoryQ', 'input': {'query': 'LangGraph framework for building language model applications'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}] additional_kwargs={} response_metadata={'id': 'msg_018d82DBs7ZKGQoevvXqjm2E', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 414, 'output_tokens': 117}} id='run-878f1b67-ef5f-4ceb-aa3f-22ea8084495e-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'LangGraph framework for building language model applications'}, 'id': 'toolu_01EKxe7qtC1ta3XokejDoryQ', 'type': 'tool_call'}] usage_metadata={'input_tokens': 414, 'output_tokens': 117, 'total_tokens': 531, 'input_token_details': {}}


[{'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph framework for building language model applications'},
  'id': 'toolu_01EKxe7qtC1ta3XokejDoryQ',
  'type': 'tool_call'}]

In [8]:
#멈춰있는데 아래의 명령어를 듣고 재시작-1
# exisiting_message를 보면 {'query': 'LangGraph for language models'} 로 LLM이 뽑은 쿼리가 합리적으로 느껴짐 그냥 두자
# None이라는 명령어를 사람이 줘서 다음으로 갈수있게된것!
events = graph.stream(None, config, stream_mode="values")

#나중에는 Tool이 주는 메세지를 내가 생성하고나 덮어쓰는법을 배움


In [9]:
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

[{'text': "Certainly! I'd be happy to research LangGraph for you. To get the most up-to-date and comprehensive information, I'll use the Tavily search engine to look this up. Let me do that for you now.", 'type': 'text'}, {'id': 'toolu_01EKxe7qtC1ta3XokejDoryQ', 'input': {'query': 'LangGraph framework for building language model applications'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01EKxe7qtC1ta3XokejDoryQ)
 Call ID: toolu_01EKxe7qtC1ta3XokejDoryQ
  Args:
    query: LangGraph framework for building language model applications
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.datacamp.com/tutorial/langgraph-tutorial", "content": "LangGraph's structured framework ensures that each agent operates efficiently and effectively, making it suitable for tasks l